In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Datasets SL-20201117T184349Z-001/Datasets SL/classified_datasets
path = "/content/drive/MyDrive/Datasets SL-20201117T184349Z-001/Datasets SL"

/content/drive/MyDrive/Datasets SL-20201117T184349Z-001/Datasets SL/classified_datasets


In [ ]:
# all library imports here
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
a_file = open("index_dictionary.pkl", "rb")
index_dictionary = pickle.load(a_file)
print(index_dictionary)

{'binary': [0, 163], 'CKSNAP1': [164, 195], 'CKSNAP3': [196, 259], 'CKSNAP5': [260, 355], 'CKSNAP7': [356, 483], 'CKSNAP9': [484, 643], 'ENAC5': [644, 791], 'ENAC10': [792, 939], 'DAC': [940, 981], '_EIIP': [982, 1022], 'PseEIIP': [1023, 1086], 'NCP': [1087, 1209], 'Kmer1': [1210, 1213], 'Kmer2': [1214, 1229], 'Kmer3': [1230, 1293], 'Kmer4': [1294, 1549], 'Kmer5': [1550, 2573], 'TAC': [2574, 2587]}


In [ ]:
import random

def getList(dict): 
    return list(dict.keys()) 

       
keyList = getList(index_dictionary)
print(type(keyList))

random_feature = [] 

for i in range(4):
  random_feature.append(random.sample(keyList, 5))

print(random_feature )

<class 'list'>
[['_EIIP', 'CKSNAP3', 'CKSNAP5', 'NCP', 'Kmer1'], ['CKSNAP3', 'ENAC5', 'DAC', 'PseEIIP', 'NCP'], ['TAC', 'Kmer4', 'binary', '_EIIP', 'CKSNAP7'], ['Kmer3', 'CKSNAP7', 'CKSNAP9', 'Kmer4', 'binary']]


In [ ]:
_train, Xa_test, ya_train, ya_test = np.load("xa_train.npy"), np.load("xa_test.npy"), np.load("ya_train.npy"), np.load("ya_test.npy")
Xc_train, Xc_test, yc_train, yc_test = np.load("xc_train.npy"), np.load("xc_test.npy"), np.load("yc_train.npy"), np.load("yc_test.npy")
Xd_train, Xd_test, yd_train, yd_test = np.load("xd_train.npy"), np.load("xd_test.npy"), np.load("yd_train.npy"), np.load("yd_test.npy")
print(Xa_train.shape, Xa_test.shape, ya_train.shape, ya_test.shape)
print(Xc_train.shape, Xc_test.shape, yc_train.shape, yc_test.shape)
print(Xd_train.shape, Xd_test.shape, yd_train.shape, yd_test.shape)

(3690, 2568) (262, 2568) (3690, 1) (262, 1)
(2898, 2568) (206, 2568) (2898, 1) (206, 1)
(3300, 2568) (234, 2568) (3300, 1) (234, 1)


In [ ]:
#pd_test = np.concatenate((pd_test, dt), axis=1)
xd_trains = []
for i, feature_groups in enumerate(random_feature):
  #no_of_features = 0
  #for j in feature_groups:
   # no_of_features = no_of_features + index_dictionary[j][1]-index_dictionary[j][0]+1
  #print(no_of_features)
  xd_train_temp = np.concatenate([Xd_train[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xd_trains.append(xd_train_temp)
  print(xd_trains[i].shape)

(3300, 328)
(3300, 441)
(3300, 589)
(3300, 772)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from tabulate import tabulate

In [ ]:
def avg(avg_list):
  return sum(avg_list)/len(avg_list)

In [ ]:
# for SVM
cv = StratifiedKFold( n_splits=5, shuffle=True)
svc = SVC() # Support vector classifier(SVC)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

raws = []
headers = ["Random-Set", "Train-Accuracy", "Test-Accuracy", "Train-Specificity", "Test-Specificity", "Train-Sensitivity", "Test-Sensitivity", "Train-MCC", "Test-MCC"]

for i, x in enumerate(xd_trains):
  for train_index, test_index in cv.split(x, yd_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yd_train[train_index], yd_train[test_index]
      svc.fit(x_train, y_train.ravel())
      test_pred = svc.predict(x_test)
      train_pred = svc.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  raws.append(["Random"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

print(" For SVM Classifier:\n")
table = tabulate(raws, headers, tablefmt='simple')
print(table)

Random set - 1
Train-Accuracy =  0.9086363636363636
Test-Accuracy =  0.8224242424242425
Train-Specificity =  0.9054545454545455
Test-Specificity =  0.8054545454545454
Train-Sensitivity =  0.9060674083743901
Test-Sensitivity =  0.8118879337469893
Train-MCC =  0.8173010938185721
Test-MCC =  0.6454304736707048



Random set - 2
Train-Accuracy =  0.9143560606060606
Test-Accuracy =  0.8224242424242423
Train-Specificity =  0.9106060606060605
Test-Specificity =  0.8054545454545454
Train-Sensitivity =  0.9112789312966058
Test-Sensitivity =  0.8120942044194603
Train-MCC =  0.8287446033820055
Test-MCC =  0.6454611828145195



Random set - 3
Train-Accuracy =  0.9218181818181818
Test-Accuracy =  0.8216161616161615
Train-Specificity =  0.9189898989898991
Test-Specificity =  0.8036363636363636
Train-Sensitivity =  0.9194843007683746
Test-Sensitivity =  0.8107286727149483
Train-MCC =  0.8436621393714342
Test-MCC =  0.644010469889263



Random set - 4
Train-Accuracy =  0.9251704545454545
Test-Accuracy

In [ ]:
###  for LogisticRegression

logisticRegression = LogisticRegression(max_iter=500)
#cv = StratifiedKFold( n_splits=5, shuffle=True)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

raws = []

for i, x in enumerate(xd_trains):
  for train_index, test_index in cv.split(x, yd_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yd_train[train_index], yd_train[test_index]
      logisticRegression.fit(x_train, y_train.ravel())
      test_pred = logisticRegression.predict(x_test)
      train_pred = logisticRegression.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  raws.append(["Random"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])
print(" For LogisticRegression\n")
table = tabulate(raws, headers, tablefmt='orgtbl')
print(table)

Random set - 1
Train-Accuracy =  0.8463636363636363
Test-Accuracy =  0.8254545454545454
Train-Specificity =  0.8353030303030302
Test-Specificity =  0.8139393939393939
Train-Sensitivity =  0.8388790963337002
Test-Sensitivity =  0.8185016625963198
Train-MCC =  0.6929079824384295
Test-MCC =  0.6515114206347066



Random set - 2
Train-Accuracy =  0.8479924242424243
Test-Accuracy =  0.8213636363636363
Train-Specificity =  0.8364393939393938
Test-Specificity =  0.8115151515151515
Train-Sensitivity =  0.8401381356390024
Test-Sensitivity =  0.8155924607558355
Train-MCC =  0.6961799875154214
Test-MCC =  0.6432822689180393



Random set - 3
Train-Accuracy =  0.848030303030303
Test-Accuracy =  0.8189898989898989
Train-Specificity =  0.8371212121212123
Test-Specificity =  0.8084848484848485
Train-Sensitivity =  0.8406109855789222
Test-Sensitivity =  0.8128500699426338
Train-MCC =  0.6962369904288346
Test-MCC =  0.6385499440430752



Random set - 4
Train-Accuracy =  0.8499431818181817
Test-Accuracy

In [ ]:
###  for XGBClassifier

xgbC = XGBClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
rows = []
for i, x in enumerate(xd_trains):
  for train_index, test_index in cv.split(x, yd_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yd_train[train_index], yd_train[test_index]
      xgbC.fit(x_train, y_train.ravel())
      test_pred = xgbC.predict(x_test)
      train_pred = xgbC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  raws.append(["Random"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

print("For XGBClassifier\n")
table = tabulate(raws, headers, tablefmt='orgtbl')
print(table)

Random set -: 1
Train-Accuracy =  0.8995454545454546
Test-Accuracy =  0.8263636363636364
Train-Specificity =  0.8915151515151516
Test-Specificity =  0.8109090909090909
Train-Sensitivity =  0.893262554778587
Test-Sensitivity =  0.8166794892434792
Train-MCC =  0.799209620581404
Test-MCC =  0.6531183191538789



Random set -: 2
Train-Accuracy =  0.9040530303030302
Test-Accuracy =  0.8253030303030304
Train-Specificity =  0.8984090909090912
Test-Specificity =  0.8133333333333332
Train-Sensitivity =  0.8995906562051129
Test-Sensitivity =  0.8177215544035918
Train-MCC =  0.8081790496446566
Test-MCC =  0.6508826738776474



Random set -: 3
Train-Accuracy =  0.9032070707070707
Test-Accuracy =  0.828080808080808
Train-Specificity =  0.8972222222222224
Test-Specificity =  0.8143434343434343
Train-Sensitivity =  0.8984722132355824
Test-Sensitivity =  0.8194409244133029
Train-MCC =  0.8064876935924555
Test-MCC =  0.6566373473411594



Random set -: 4
Train-Accuracy =  0.9028219696969698
Test-Accura

In [ ]:
###  for AdaBoostClassifier

adaBC = AdaBoostClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
rows = []

for i, x in enumerate(xd_trains):
  for train_index, test_index in cv.split(x, yd_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = yd_train[train_index], yd_train[test_index]
      adaBC.fit(x_train, y_train.ravel())
      test_pred = adaBC.predict(x_test)
      train_pred = adaBC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  raws.append(["Random"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

print("For AdaBoostClassifier\n")
table = tabulate(raws, headers, tablefmt='orgtbl')
print(table)

Random set -: 1
Train-Accuracy =  0.8349242424242425
Test-Accuracy =  0.8027272727272727
Train-Specificity =  0.8262121212121212
Test-Specificity =  0.7915151515151515
Train-Sensitivity =  0.8292070489486809
Test-Sensitivity =  0.7964780349379109
Train-MCC =  0.6699608756583341
Test-MCC =  0.6061549643689264



Random set -: 2
Train-Accuracy =  0.8371590909090909
Test-Accuracy =  0.8042424242424241
Train-Specificity =  0.8308333333333333
Test-Specificity =  0.7933333333333333
Train-Sensitivity =  0.8330084537819917
Test-Sensitivity =  0.7979018647515802
Train-MCC =  0.6744009046561031
Test-MCC =  0.6090339561396739



Random set -: 3
Train-Accuracy =  0.8376262626262625
Test-Accuracy =  0.804141414141414
Train-Specificity =  0.8290909090909092
Test-Specificity =  0.7943434343434344
Train-Sensitivity =  0.8320077537652707
Test-Sensitivity =  0.7986378115902999
Train-MCC =  0.6753844972457038
Test-MCC =  0.6087908731364226



Random set -: 4
Train-Accuracy =  0.8371780303030303
Test-Accu